In [1]:
import torch 
import torchvision 
import numpy as np

import os
import pandas as pd
import albumentations as A
import albumentations.pytorch
import cv2

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
# import torch.nn.functional as F

import sys
sys.path.append('../../')
from dataset import val_transforms, CDataset

In [2]:
import easydict 
args = easydict.EasyDict({ "batch_size": 256, 
                          "epochs": 100, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':2,
                          'workers':8,
                         'print_freq':1000,
                         'saved_dir':'../../trained_models/stain1/checkpoint.pt'})

In [3]:
# make saved dir
from pathlib import Path
path = Path(args.saved_dir.split('checkpoint')[0])
path.mkdir(parents=True, exist_ok=True)

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]= "7"
# device = torch.device("cuda")

# print('Device:', device)
# print('Current cuda device:', torch.cuda.current_device())
# print('Count of using GPUs:', torch.cuda.device_count())

In [5]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
# device = torch.device('cpu')
# device = torch.device('cuda')
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

6
cuda:2


In [6]:
IMAGE_SIZE = 256
from augment import HEColorAugment
train_transforms = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1),
    A.OneOf([
        A.HorizontalFlip(p=.8),
        A.VerticalFlip(p=.8),
        A.RandomRotate90(p=.8)]
    ),
    A.RandomResizedCrop(height=IMAGE_SIZE,width=IMAGE_SIZE,scale=[0.95,1.05],ratio=[0.95,1.05],p=0.5),
    A.transforms.ColorJitter(brightness=0.05, contrast=0.2, saturation=0.2, hue=0.02, p=.8),
    HEColorAugment(sigma1=0.05, sigma2=3, theta=9, p=.8),
], p=1.0) 

In [7]:
train_df = pd.read_csv('../../dataframe/train_df.csv')
train_dataset = CDataset(train_df, transform=train_transforms)

val_df = pd.read_csv('../../dataframe/val_df.csv')
val_dataset = CDataset(val_df, transform=val_transforms)  

test_df = pd.read_csv('../../dataframe/test_df.csv')
test_dataset = CDataset(test_df, transform=val_transforms)  

In [8]:
image, label, path = next(iter(train_dataset))

In [9]:
train_loader = DataLoader(train_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)

val_loader = DataLoader(val_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)

test_loader = DataLoader(test_dataset, batch_size=args.batch_size,
                          shuffle=True, num_workers=args.workers)


In [10]:
images, labels, paths = next(iter(train_loader))
images.shape

torch.Size([256, 3, 256, 256])

In [11]:
import torchvision.models as models
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 3)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 30, 40, 50], 
                                                    gamma=0.2)
# criterion = nn.CrossEntropyLoss(label_smoothing=0.0)
criterion = nn.CrossEntropyLoss().to(device)

In [12]:
from main import *

best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []

for epoch in range(60):
    losses = train(train_loader, model, criterion, optimizer, epoch, args)
    scheduler.step()

    # evaluate after every epoch
    acc1 = validate(val_loader, model, criterion, args)    
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1) 
    
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
        'scheduler' : scheduler.state_dict(),
    }, is_best, filename=args.saved_dir)    
    

Epoch: [0][   0/1121]	Time  4.151 ( 4.151)	Data  2.647 ( 2.647)	Loss 9.6864e+00 (9.6864e+00)	Acc@1   0.00 (  0.00)
Epoch: [0][1000/1121]	Time  0.411 ( 0.393)	Data  0.052 ( 0.064)	Loss 5.8378e-01 (1.0367e+00)	Acc@1  75.00 ( 57.23)
Test: [  0/440]	Time  1.986 ( 1.986)	Loss 8.6677e-01 (8.6677e-01)	Acc@1  68.36 ( 68.36)
 * Acc@1 70.464
Epoch: [1][   0/1121]	Time  3.741 ( 3.741)	Data  3.517 ( 3.517)	Loss 5.4039e-01 (5.4039e-01)	Acc@1  76.17 ( 76.17)
Epoch: [1][1000/1121]	Time  0.374 ( 0.392)	Data  0.071 ( 0.068)	Loss 3.3150e-01 (4.1952e-01)	Acc@1  87.50 ( 83.61)
Test: [  0/440]	Time  1.998 ( 1.998)	Loss 5.3732e-01 (5.3732e-01)	Acc@1  82.81 ( 82.81)
 * Acc@1 83.514
Epoch: [2][   0/1121]	Time  3.095 ( 3.095)	Data  2.853 ( 2.853)	Loss 3.2386e-01 (3.2386e-01)	Acc@1  87.11 ( 87.11)
Epoch: [2][1000/1121]	Time  0.382 ( 0.396)	Data  0.060 ( 0.071)	Loss 2.9228e-01 (3.1917e-01)	Acc@1  90.62 ( 87.64)
Test: [  0/440]	Time  1.975 ( 1.975)	Loss 1.1203e+00 (1.1203e+00)	Acc@1  73.05 ( 73.05)
 * Acc@1 68.55

In [13]:
checkpoint = torch.load(args.saved_dir)
model.load_state_dict(checkpoint['state_dict'])
acc1 = validate(test_loader, model, criterion, args)    

Test: [  0/266]	Time  5.567 ( 5.567)	Loss 3.4405e-01 (3.4405e-01)	Acc@1  91.80 ( 91.80)
 * Acc@1 90.537
